In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_absolute_error
from pmdarima.arima import auto_arima
from pmdarima.arima import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
from fbprophet import Prophet
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pickle
from sklearn.ensemble import RandomForestRegressor

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dropout

In [56]:
humedad = pd.read_csv('humedad_suelo_1.csv')

salinidad = pd.read_csv('salinidad_suelo_1.csv')

temperatura = pd.read_csv('temperatura_suelo_1.csv')

ambiente = pd.read_csv('temperatura_humedad_1.csv')

In [57]:
salinidad['Fecha'] = pd.to_datetime(salinidad.Fecha)

humedad['Fecha'] = pd.to_datetime(humedad.Fecha)

temperatura['Fecha'] = pd.to_datetime(temperatura.Fecha)

ambiente['Fecha'] = pd.to_datetime(ambiente.Fecha)

In [58]:
salinidad = salinidad.set_index('Fecha')

humedad = humedad.set_index('Fecha')

temperatura = temperatura.set_index('Fecha')

ambiente = ambiente.set_index('Fecha')

In [59]:
# Primero eliminamos las profundidades de 10, 70 y 80 cm

temperatura = temperatura.drop(['T10', 'T70', 'T80'], axis=1)

salinidad = salinidad.drop(['S10', 'S70', 'S80'], axis=1)

humedad = humedad.drop(['H10', 'H70', 'H80'], axis=1)

In [60]:
# Segundo sacamos la media 

temperatura['T'] = temperatura.mean(axis=1)

salinidad['S'] = salinidad.mean(axis=1)

humedad['H'] = humedad.mean(axis=1)

In [61]:
temp = temperatura.loc[:,'T']

sal = salinidad.loc[:,'S']

hum = humedad.loc[:,'H']

hum

Fecha
2021-07-01 00:08:29    44.726
2021-07-01 00:18:46    44.722
2021-07-01 00:29:03    44.702
2021-07-01 00:39:20    44.712
2021-07-01 00:49:37    44.690
                        ...  
2021-12-31 23:09:30    43.252
2021-12-31 23:19:47    43.268
2021-12-31 23:30:04    43.260
2021-12-31 23:40:21    43.264
2021-12-31 23:50:38    43.270
Name: H, Length: 25050, dtype: float64

In [73]:
temperatura_pro = temp.reset_index()

salinidad_pro = sal.reset_index()

humedad_pro = hum.reset_index()

temperatura_pro



,Fecha,T
0,2021-07-01 00:08:29,22.396
1,2021-07-01 00:18:46,22.392
2,2021-07-01 00:29:03,22.384
3,2021-07-01 00:39:20,22.374
4,2021-07-01 00:49:37,22.354
...,...,...
25045,2021-12-31 23:09:30,10.782
25046,2021-12-31 23:19:47,10.776
25047,2021-12-31 23:30:04,10.764
25048,2021-12-31 23:40:21,10.754


In [74]:
temperatura_pro = temperatura_pro.resample('H', on='Fecha').mean().dropna()
salinidad_pro = salinidad_pro.resample('H', on='Fecha').mean().dropna()
humedad_pro = humedad_pro.resample('H', on='Fecha').mean().dropna()

In [75]:
from sklearn.preprocessing import MinMaxScaler

train = salinidad_pro.reset_index()

In [76]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

n_input = 72
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)

In [77]:
from tensorflow.keras.optimizers import Adam

In [78]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(32, activation="relu"))
model.add(Dense(1))
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')
history = model.fit_generator(generator,epochs=50,verbose=1)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_15344\2455045798.py:7: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



KeyError: 72

In [ ]:
pred_list = []
batch = train[-n_input:].reshape((1, n_input, n_features))
for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

1/1 [==============================] - 0s 32ms/step


In [ ]:
batch

array([[[0.15029649],
        [0.15321456],
        [0.15957016],
        [0.16829814],
        [0.17863202],
        [0.18997493],
        [0.20182864],
        [0.21360645],
        [0.22504941],
        [0.23589599],
        [0.24601786],
        [0.25538301],
        [0.26403981],
        [0.2720972 ],
        [0.2796281 ],
        [0.28671524],
        [0.29344153],
        [0.29988465],
        [0.30605957],
        [0.31194347],
        [0.31753898],
        [0.32286602],
        [0.32793117],
        [0.33276835],
        [0.3374019 ],
        [0.34185633],
        [0.34615344],
        [0.35031131],
        [0.35434413],
        [0.35826269],
        [0.36207476],
        [0.36578596],
        [0.36940011],
        [0.37291998],
        [0.37634754],
        [0.37968442],
        [0.38293216],
        [0.38609233],
        [0.38916659],
        [0.3921569 ],
        [0.39506537],
        [0.39789414],
        [0.4006457 ],
        [0.4033632 ],
        [0.40603739],
        [0

In [ ]:
from pandas.tseries.offsets import DateOffset

In [ ]:
add_dates = [temperatura_pro.index[-1] + DateOffset(hours=x) for x in range(0,73) ]
future_dates = pd.DataFrame(index=add_dates[1:],columns=temperatura_pro.columns)

In [ ]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['Prediction'])
df_proj = pd.concat([temperatura_pro,df_predict], axis=1)

In [ ]:
df_proj

,T,Prediction
2021-07-01 00:00:00,22.373000,NaN
2021-07-01 01:00:00,22.316800,NaN
2021-07-01 02:00:00,22.243000,NaN
2021-07-01 03:00:00,22.172333,NaN
2021-07-01 04:00:00,22.085333,NaN
...,...,...
2022-01-03 19:00:00,NaN,15.665806
2022-01-03 20:00:00,NaN,15.689784
2022-01-03 21:00:00,NaN,15.713221
2022-01-03 22:00:00,NaN,15.736129


In [ ]:
plot_data = [
    go.Scatter(
        x=df_proj.index,
        y=df_proj['T'],
        name='actual'
    ),
    go.Scatter(
        x=df_proj.index,
        y=df_proj['Prediction'],
        name='prediction'
    )
]
plot_layout = go.Layout(
        title='Temperature prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
iplot(fig)